# Search Engine

1. Connectors
2. Indexation
3. HashMap
4. Save
5. Query
    * search(word)
    * search word0 AND word1 AND word2
    * search word0 OR word1 OR word2

### Deps

> Python3 (Python 3.6.8)  

In [1]:
# Mettre le path vers le dossier pour fetch
path = "/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism"

### Connecteurs

File system: prendre tous les fichier lisisbles d'un répertoire

In [24]:
from os import listdir
from os.path import isfile, isdir, join

class Doc:
    def __init__(self, url, text):
        self.text = text # Raw strnig of the whole text  
        self.url = url
    pass

def force_to_unicode(text):
    "If text is unicode, it is returned as is. If it's str, convert it to Unicode using UTF-8 encoding"
    return text if isinstance(text, unicode) else text.decode('utf8')

def fetch(path, recursive=True):
    DocList = []
    
    files = [f for f in listdir(path) if isfile(join(path, f))]
    dirs  = [d for d in listdir(path) if isdir(join(path, d))]
    
    for fname in files:
        f = open(path + "/" + fname, "r", errors="ignore")
        try:
            c = f.read()
            DocList.append(Doc(path + "/" + fname, c))
        except Exception:
            pass
        
        f.close
    
    if recursive:
        for d in dirs:
            DocList += fetch(join(path, d), recursive=recursive)              
    
    return DocList


# res = fetch("/home/sidore_m/projects/search_engine/data/20news-bydate-train/")

DocList = fetch(path)

# Debug
print(len(DocList))
# (DocList[0].url, DocList[0].text)

480


### Analyseur

* Transformer le texte brut des documents d'entrées en mots (effectuer des traitements sur ces mots)

In [3]:
class LowerProcessor:
    def __init__(self):
        pass
    def process(word):
        """
        word      - string
        return    - processed string
        """
        proc_word = word.lower()
        return proc_word
    
class AccentProcessor:
    def __init__(self):
        pass
    def process(word):
        """
        word      - string
        return    - processed string
        """
        proc_word = word.replace("'", " ")
        proc_word = proc_word.replace("`", " ")
        proc_word = proc_word.replace("\"", "")
        return proc_word
    
class PunctationProcessor:
    def __init__(self):
        pass
    def process(word):
        """
        word      - string
        return    - processed string
        """
        proc_word = word.replace(".", " ")
        proc_word = proc_word.replace(",", " ")
        proc_word = proc_word.replace(";", "")
        proc_word = proc_word.replace(":", "")
        proc_word = proc_word.replace("!", "")
        proc_word = proc_word.replace("?", "")
        proc_word = proc_word.replace("\n", " ")
        return proc_word
    
class SpecialProcessor:
    def __init__(self):
        pass
    def process(word):
        """
        word      - string
        return    - processed string
        """
        proc_word = word.replace("(", "")
        proc_word = proc_word.replace("-", " ")
        proc_word = proc_word.replace("_", " ")
        proc_word = proc_word.replace("=", " ")
        proc_word = proc_word.replace("*", " ")
        proc_word = proc_word.replace("<", "")
        proc_word = proc_word.replace(">", "")
        proc_word = proc_word.replace(")", "")
        proc_word = proc_word.replace("/", "")
        proc_word = proc_word.replace("\\", "")
        proc_word = proc_word.replace("[", "")
        proc_word = proc_word.replace("^", "")
        proc_word = proc_word.replace("]", "")
        proc_word = proc_word.replace("", "")
        proc_word = proc_word.replace(" ", "")
        proc_word = proc_word.replace("  ", "")
        return proc_word     

class TokenizedDoc:
    def __init__(self, words, url):
        self.words = words # list of strings (list of words)
        self.url = url
    pass    


def analyse(docs, processorsList):
    """
    docs           - list of Docs
    processorsList - list of Processors like TextProcessor
    return         - list of TokenizedDoc
    """
    tokdocs = []
    
    for d in docs:
        tokens = d.text.split(" ") # Split and tokenzie on ' ', return list of words
        # Process
        words = []
        for w in tokens:
            w = w.strip()
            for p in processorsList:
                 w = p.process(w)
            w_r = w.split()
            for w in w_r:
                if w != '' and w != ' ':        
                    words.append(w)
        tokdocs.append(TokenizedDoc(words, d.url))    
    
    return tokdocs
    
Processors = [PunctationProcessor, SpecialProcessor, AccentProcessor, LowerProcessor]
                                      
TokDocs = analyse(DocList, Processors)

len(TokDocs)
print(TokDocs[0].words[0], TokDocs[0].words[1], TokDocs[0].words[2],
      TokDocs[1].words[2], TokDocs[2].words[30], TokDocs[2].words[45])


from rashidsubject re bill you have


In [23]:
# for KKK in TokDocs:
#     print(KKK.words)

### Indexeur

* Transforme les listes de mots appartenant au document en des listes inversées de mots associés à des documents 

In [5]:
class Posting:
    def __init__(self, word, urls):
        """
        word - string
        urls - list of url
        """
        self.word = word
        self.urls = urls
    pass

def make_index(TokeneizedDocs):
    """
    TokeneizedDocs - list of TokenizedDoc
    """
    done_words = []
    postingList = []
    
    words = []
    for tokD in TokeneizedDocs:
        words += tokD.words
        
    print("Number of words ", len(words))    
    
        
    for w in words:
        if w in done_words:
            continue
                
        done_words.append(w)
        urls = []
        for d in TokeneizedDocs:
            if w in d.words:
                urls.append(d.url)
        postingList.append(Posting(w, urls))   
    
    return postingList

class Index:
    def __init__(self, urlToDocId, wordToDocIds, idToUrl):
        """
        urlToDocId   - Map/Dico <string, int> 
        wordToDocIds - Map/Dico <string, int[]> 
        idToUrl   - Map/Dico <int, string> 
        """
        self.urlToDocId = urlToDocId
        self.idToUrl = idToUrl
        self.wordToDocIds = wordToDocIds
    pass


def build(Postings):
    """
    Build Index from list of Posting
    """
    urlToId = {}
    idToUrl = {}
    wordToDocIds = {}
    ids = 0
    for i in Postings:
        u_list = []
        for j in i.urls:
            if not (j in urlToId):
                urlToId[j] = ids
                idToUrl[ids] = j
                u_list.append(ids)
                ids += 1
            else:
                u_list.append(urlToId[j])
        wordToDocIds[i.word] = u_list     
        
    return Index(urlToId, wordToDocIds, idToUrl)



postingList = make_index(TokDocs)
index = build(postingList)

print(len(TokDocs), len(postingList))
print(len(index.urlToDocId), len(index.wordToDocIds))

Number of words  151248
480 19906
480 19906


In [6]:
import pickle

path_bin_index = "/home/sido/projects/SearchEngine/"

def save(index, path):
    """
    Save index on disk
    usin pickle python lib
    """
    with open(path + "/" + "index.bin", "wb") as f:
        pickle.dump(index, f, pickle.HIGHEST_PROTOCOL)
        
save(index, path_bin_index)

def load(path):
    with open(path + "/" + "index.bin", "rb") as f:
        index = pickle.load(f)
    return index

index = load(path_bin_index)
print(len(index.urlToDocId), len(index.wordToDocIds))

480 19906


### Searcher

* Lire les listes inversées afin de répondre à une requete
* Pouvoir lire un index sauvegardé précédemment sur disque


In [7]:
# an object 'Index' as index should exist !

class Searcher:
    def __init__(self):
        self.index = None
    
    def load(self, path):
        with open(path + "/" + "index.bin", "rb") as f:
            self.index = pickle.load(f)
            
    
    def search(self, word, nb_urls=50):
        """
        Search word, return urls for this word
        word      - string
        nb_urls   - number of result urls to give back, if 0 return all
        index_obj - specify an indexobject
        """
        if not (word in self.index.wordToDocIds):
            return "No results, word not indexed"
        urls = self.index.wordToDocIds[word]
        res = ""
        for k in urls:
            res += self.index.idToUrl[k] + "\n"
        return res


    def searchAllOf(self, words):
        """
        AND oper
        words - list of words
        """
        and_urls = []
        first = True
        for w in words:
            if not (w in self.index.wordToDocIds):
                return "No results, one of the words is not indexed"
            urls = self.index.wordToDocIds[w]
            if first:
                and_urls = urls
                first = False
            else:
                and_urls = list(set(and_urls) & set(urls))
        res = ""
        for k in and_urls:
            res += self.index.idToUrl[k] + "\n"
        return res

    def searchOneOf(self, words):
        """
        OR oper
        """
        or_urls = []
        for w in words:
            if not (w in self.index.wordToDocIds):
                return "No results, one of the words is not indexed"
            urls = self.index.wordToDocIds[w]
            or_urls = list(set(or_urls) | set(urls))
        res = ""
        for k in or_urls:
            res += self.index.idToUrl[k] + "\n"
        return res



In [8]:
searcher = Searcher()
searcher.load(path_bin_index)

print(searcher.search('sports'))

/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/53090
/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/53373
/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/53136
/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/53521



In [9]:
print(searcher.searchAllOf(['sports', 'is']))

/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/53521
/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/53136
/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/53373
/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/53090



In [10]:
print(searcher.searchOneOf(['sports', 'and', 'of']))

/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/53120
/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/51317
/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/51201
/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/51318
/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/53229
/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/53177
/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/53230
/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/53150
/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/53067
/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/53434
/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/51285
/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/53466
/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/53440
/home/sido/p

### Indexation Incrementale

In [11]:
class Generation:
    def __init__(self, gen_id, wordToDocIds):
        self.gen_id = gen_id
        self.wordToDocIds = wordToDocIds

class IndexIncr:
    def __init__(self):
        self.ids = 0
        self.urlToDocId = {}
        self.idToDocUrl = {}
        self.docIdToGeneration = {}  # key (gen) = [doc id, ...]
        self.generations = []
        # Bonus 2 : Suppr vector
        self.suppressions = []
    
    def build_gen(self, Postings):
        """
        Build Index from list of Posting
        """
        gen_id = len(self.generations)
        wordToDocIds = {}
        for i in Postings:
            u_list = []
            for j in i.urls:
                if not (j in self.urlToDocId):
                    self.urlToDocId[j] = self.ids
                    self.idToDocUrl[self.ids] = j
                    self.docIdToGeneration[self.ids] = gen_id
                    u_list.append(self.ids)
                    self.ids += 1
                else:
                    u_list.append(self.urlToDocId[j])
            wordToDocIds[i.word] = u_list        
        self.generations.append(Generation(gen_id, wordToDocIds))
    


In [12]:
main_index = IndexIncr()

main_index.build_gen(postingList)

path_politics = "/home/sido/projects/SearchEngine/data/20news-bydate-train/talk.politics.guns"

path_sci_space = "/home/sido/projects/SearchEngine/data/20news-bydate-train/sci.space/"

DocList1 = fetch(path_politics)
DocList2 = fetch(path_sci_space)

TokDocs1 = analyse(DocList1, Processors)
TokDocs2 = analyse(DocList2, Processors)

postingList1 = make_index(TokDocs1)
postingList2 = make_index(TokDocs2)



Number of words  182486
Number of words  163558


In [13]:
main_index.build_gen(postingList1)

In [15]:
main_index.build_gen(postingList2)

In [16]:
import pickle

path_bin_index = "/home/sido/projects/SearchEngine/"

def save_incr(index, path):
    """
    Save index on disk
    usin pickle python lib
    """
    with open(path + "/" + "index_incr.bin", "wb") as f:
        pickle.dump(index, f, pickle.HIGHEST_PROTOCOL)
        
save_incr(main_index, path_bin_index)

def load_incr(path):
    with open(path + "/" + "index_incr.bin", "rb") as f:
        index = pickle.load(f)
    return index

index_incr = load_incr(path_bin_index)
index_incr

In [17]:
# For the SearchIncrIndex, I prefer retruning python list  as result,
# and not like previously a string

class SearchIncrIndex:
    def __init__(self, index_incr=None):
        if None == index_incr:
            self.load(path_bin_index)
        else:
            self.index = index_incr
    
    def load(self, path):
        with open(path + "/" + "index_incr.bin", "rb") as f:
            self.index = pickle.load(f)
    
    def id_to_url(self, id_l):
        res = []
        for i in id_l:
            res.append(self.index.idToDocUrl[i])
        return res
    
    def search(self, word):
        res_id = self.search_ids(word)
        res_urls = self.id_to_url(res_id)
        return res_urls
    
    def search_ids(self, word):
        """
        Search word, return urls for this word
        word      - string
        """
        gs = self.index.generations
        res = []
        gl = len(self.index.generations)
        current_urls = []
        for g in reversed(gs):  # Get last first
            if not (word in g.wordToDocIds):
                continue
            preserved_urls = []
            urls_ids = list.copy(g.wordToDocIds[word])
            urls = []
            for u in urls_ids:
                urls.append(self.index.idToDocUrl[u])
            for u in urls:
                if not (u in current_urls):
                    preserved_urls.append(u)
                    current_urls.append(u)
            urls_ids = []
            for u in preserved_urls:
                urls_ids.append(self.index.urlToDocId[u])
            res = res + urls_ids
        return res


    def searchAllOf(self, words):
        """
        AND oper
        words - list of words
        """
        and_urls = []
        first = True
        for w in words:
            if first:
                and_urls = self.search_ids(w)
                first = False
            else:
                and_urls = list(set(and_urls) & set(self.search_ids(w)))
        res_urls = self.id_to_url(and_urls)
        return res_urls

    def searchOneOf(self, words):
        """
        OR oper
        """
        or_urls = []
        for w in words:
            or_urls = list(set(or_urls) | set(self.search_ids(w)))
        res_urls = self.id_to_url(or_urls)
        return res_urls


In [18]:
incr_searcher = SearchIncrIndex(index_incr)

In [19]:
incr_searcher.search('sports')

['/home/sido/projects/SearchEngine/data/20news-bydate-train/talk.politics.guns/54215',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/talk.politics.guns/54690',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/talk.politics.guns/54235',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/53090',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/53373',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/53136',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/53521']

In [20]:
incr_searcher.searchAllOf(['sports', 'is'])

['/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/53521',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/talk.politics.guns/54235',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/53136',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/talk.politics.guns/54215',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/53090',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/talk.politics.guns/54690',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/53373']

In [21]:
incr_searcher.searchOneOf(['sports', 'and', 'of'])

['/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/53120',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/51317',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/51201',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/51318',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/53229',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/53177',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/53230',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/53150',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/53067',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/53434',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/51285',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/53466',
 '/home/sido/projects/SearchEngine/data/

In [22]:
index_incr.generations

In [34]:
DocList3 = fetch(path)

TokDocs3 = analyse(DocList3, Processors)

postingList3 = make_index(TokDocs3)

Number of words  150896


In [35]:
index_incr.build_gen(postingList3)

In [36]:
incr_searcher.searchAllOf(['sports', 'from'])

['/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/53521',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/talk.politics.guns/54235',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/53136',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/talk.politics.guns/54215',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/53090',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/talk.politics.guns/54690',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/53373']

In [37]:
incr_searcher.searchAllOf(['from'])

['/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/53120',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/51317',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/51201',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/51318',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/53229',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/53177',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/53230',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/53150',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/53067',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/53434',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/51285',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/53466',
 '/home/sido/projects/SearchEngine/data/

In [38]:
incr_searcher = SearchIncrIndex(index_incr)
incr_searcher.search('sports')

['/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/53090',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/53373',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/53136',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/53521',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/talk.politics.guns/54215',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/talk.politics.guns/54690',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/talk.politics.guns/54235']

In [39]:
incr_searcher.searchAllOf(['sports', 'from'])

['/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/53521',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/talk.politics.guns/54235',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/53136',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/talk.politics.guns/54215',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/53090',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/talk.politics.guns/54690',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/53373']

In [40]:
incr_searcher.searchAllOf(['from', 'sido'])

['/home/sido/projects/SearchEngine/data/20news-bydate-train/alt.atheism/51220']

In [41]:
incr_searcher.searchAllOf(['from', 'weapons'])

['/home/sido/projects/SearchEngine/data/20news-bydate-train/talk.politics.guns/54314',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/talk.politics.guns/54316',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/talk.politics.guns/54191',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/sci.space//60184',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/talk.politics.guns/54173',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/talk.politics.guns/54436',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/talk.politics.guns/54406',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/talk.politics.guns/54231',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/talk.politics.guns/54525',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/talk.politics.guns/53299',
 '/home/sido/projects/SearchEngine/data/20news-bydate-train/talk.politics.guns/54681',
 '/home/sido/projects/SearchEngine/data/20news-byda